In [1]:
import pandas as pd
import numpy as np 
import os, sys
from tqdm.notebook import tqdm
import itertools as it
import matplotlib as mpl
import matplotlib.pyplot as plt
from rdkit import Chem
import sys
sys.path.append("reinvent/")
import pandas as pd
import numpy as np 
import scipy.stats as sps
from tqdm.auto import tqdm, trange
from models import dataset
from models.model import Model
from utils.smiles import standardize_smiles
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=False, verbose=1)
chembl_prior = Model.load_from_file("reinvent/data/augmented.prior")

In [2]:
data_in = pd.read_table("data/pubchem_reinvent_assay_low_hitrate_human_interactions.tsv.gz", sep="\t")

In [3]:
data_in

,Unnamed: 0,aid,sid,gi_number,result,cid,NostereoAromaticSMILES,Entry
0,268,1779,842121,224586929,0,6603008,CCOCCCNCC(=O)Nc1ccc(cc1)OC(F)(F)F,Q9Y2R2
1,269,1779,842122,224586929,0,6602571,COCCn1c(nnn1)CN2CCC(CC2)Cc3ccccc3,Q9Y2R2
2,270,1779,842123,224586929,0,6602616,COCCn1c(nnn1)CN2CCC(CC2)(c3cccc(c3)C(F)(F)F)O,Q9Y2R2
3,271,1779,842124,224586929,0,644371,c1cc(ccc1CCNC(=O)CN2CCCCCC2)F,Q9Y2R2
4,272,1779,842125,224586929,0,6603132,COc1ccc(cc1)C(=O)C(c2ccccc2)N3CCOCC3,Q9Y2R2
...,...,...,...,...,...,...,...,...
80838259,160788292,743192,162211043,291575128,0,3474778,COc1ccc(cc1)n2cnc3c2ccc(c3)C(=O)N4CCCCC4,P07195
80838260,160788372,743374,172236216,119613085,0,72737738,COc1ccc(cc1)CN2CCOC(C2)CCc3ccccc3,P21918
80838261,160788373,743374,172236217,119613085,0,72737723,c1ccc(cc1)CCC2CN(CCO2)Cc3ccc(cc3)Cl,P21918
80838262,160788376,743378,172236216,49456799,0,72737738,COc1ccc(cc1)CN2CCOC(C2)CCc3ccccc3,Q6FH34


In [4]:
assays_per_target = data_in.pivot_table(index=["NostereoAromaticSMILES", "Entry"],
                                        columns="result",
                                        values="aid",
                                        aggfunc="nunique",
                                        fill_value=0)
assays_per_target.rename(columns={0: "inactive", 1: "active"},inplace=True)

target_count = data_in.pivot_table(index="NostereoAromaticSMILES", 
                                   columns="result", 
                                   values="Entry", 
                                   aggfunc="nunique", 
                                   fill_value=0)
target_count.rename(columns={0: "inactive", 1: "active"},inplace=True)


assay_count = data_in.pivot_table(index="NostereoAromaticSMILES", 
                                   columns="result", 
                                   values="aid", 
                                   aggfunc="nunique", 
                                   fill_value=0)
assay_count.rename(columns={0: "inactive", 1: "active"},inplace=True)

In [5]:
target_assay_count = target_count.join(assay_count, lsuffix= " targets", rsuffix= " assays")

In [6]:
assert(len(assays_per_target.query("active >= 1 and inactive >= 1")) == 0)

In [7]:
def encodable(smiles, voc=chembl_prior.vocabulary, tokenizer=chembl_prior.tokenizer):
    try:
        tokens = tokenizer.tokenize(smiles)
        encoded = voc.encode(tokens)
        if len(encoded) > 2:
            return True
        else:
            return False
    except:
        return False

In [ ]:
smiles_cid = data_in.groupby("NostereoAromaticSMILES")
smiles_cid = pd.DataFrame(smiles_cid["cid"].agg(set))
smiles_cid["NostereoAromaticSMILES"] = smiles_cid.index
smiles_cid["rdkit_smiles"] = smiles_cid["NostereoAromaticSMILES"].parallel_map(standardize_smiles)
smiles_cid["encodable"] = smiles_cid["rdkit_smiles"].parallel_map(encodable)

In [ ]:
assert(len(smiles_cid) == len(smiles_cid.groupby("rdkit_smiles")))
assert(len(smiles_cid) == len(smiles_cid.query("encodable == True")))

In [ ]:
smiles_aid = data_in.groupby("NostereoAromaticSMILES")
smiles_aid = pd.DataFrame(smiles_aid["aid"].agg(set))
smiles_Entry = data_in.groupby("NostereoAromaticSMILES")
smiles_Entry = pd.DataFrame(smiles_Entry["Entry"].agg(set))

In [ ]:
target_count = target_count.join(smiles_cid[["rdkit_smiles", "cid", "encodable"]]).join(smiles_aid).join(smiles_Entry)
assay_count = assay_count.join(smiles_cid[["rdkit_smiles", "cid", "encodable"]]).join(smiles_aid).join(smiles_Entry)
target_assay_count = target_assay_count.join(smiles_cid[["rdkit_smiles", "cid", "encodable"]]).join(smiles_aid).join(smiles_Entry)

In [ ]:
def data_sets_available():
    paths = ["data/pubchem_assay_compounds_processed_training.smi", "data/pubchem_assay_compounds_processed.tsv", "data/all_extracted_pubchem_mt_st_nt_compounds.tsv.gz"]
    for path in paths:
        if not os.path.exists(path):
            return False
    return True 

if not data_sets_available():
    st = target_count.query("active == 1 & inactive >= 4").copy()
    mt = target_count.query("active >= 5").copy()
    nt = target_count.query("active == 0 & inactive >= 5").sample(n=len(st)).copy()

    st["class"] = "singletarget"
    st["set"] = "test"
    mt["class"] = "multitarget"
    mt["set"] = "test"
    nt["class"] = "notarget"
    nt["set"] = "test"

    training_mt = mt.sample(n=1000).index
    mt.loc[training_mt,"set"] = "training"
    dataset = pd.concat([mt, st, nt])

    dataset["cids"] = dataset["cid"].map(lambda x: ",".join(sorted(map(str,x))))
    dataset["aids"] = dataset["aid"].map(lambda x: ",".join(sorted(map(str,x))))
    dataset["Targets"] = dataset["Entry"].map(lambda x: ",".join(sorted(map(str,x))))

    if not os.path.exists("data/pubchem_assay_compounds_processed_training.smi"):
        dataset.query("set == 'training'").sample(frac=1.)[["rdkit_smiles"]].to_csv("data/pubchem_assay_compounds_processed_training.smi",index=None,header=None)

    if not os.path.exists("data/pubchem_assay_compounds_processed.tsv"):
        dataset = dataset.reset_index()[['cids', 'NostereoAromaticSMILES', 'rdkit_smiles', 'class', 'set', 'inactive', 'active', 'encodable']]
        dataset.to_csv("data/pubchem_assay_compounds_processed.tsv",index=None, sep="\t")
        
    if not os.path.exists("data/all_extracted_pubchem_mt_st_nt_compounds.tsv.gz"):
        nt_all = target_count.query("active == 0 & inactive >= 5 & index not in @nt").copy()
        nt_all["class"] = "notarget"
        nt_all["set"] = "unused"

        dataset_all = pd.concat([mt, st, nt, nt_all])
        dataset_all['index_num'] = range(len(dataset_all))
        dataset_all = dataset_all[["class", "set", "index_num"]].join(target_assay_count)
        dataset_all = dataset_all[["rdkit_smiles", "cid", "class", "set", "aid", "active assays", "inactive assays", "Entry", "active targets", "inactive targets", "index_num"]].copy()

        dataset_all["cid"] = dataset_all["cid"].map(lambda x: ",".join(sorted(map(str,x))))
        dataset_all["aid"] = dataset_all["aid"].map(lambda x: ",".join(sorted(map(str,x))))
        dataset_all["Entry"] = dataset_all["Entry"].map(lambda x: ",".join(sorted(map(str,x))))
        
        dataset_all = dataset_all.sort_values("index_num")
        del dataset_all["index_num"]
        dataset_all = dataset_all.rename(mapper={"cid": "cids", "aid": "validated assay ids","Entry": "validated target ids"}, axis=1).reset_index()
        
        dataset_all.to_csv("data/all_extracted_pubchem_mt_st_nt_compounds.tsv.gz", sep="\t")